# Demo Notebook:
## Survival Transformer For Causal Sequence Modelling 

Including time, and excluding tabular values

In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-env-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

!pwd

%load_ext autoreload
%autoreload 2

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-env-icelake/lib/python3.10/site-packages' at start of search paths.
/rds/homes/g/gaddcz/Projects/CPRD/examples/modelling/SurvStreamGPT


In [2]:
import pytorch_lightning
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
import sqlite3
from dataclasses import dataclass
import logging
from CPRD.data.foundational_loader import FoundationalDataModule
from CPRD.src.models.survival.task_heads.causal import SurvStreamGPTForCausalModelling
from pycox.evaluation import EvalSurv

# TODO:
# replace experiment boilerplate with pytorch lightning

torch.manual_seed(1337)
logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"    # if more informative debugging statements are needed
print(f"Using device: {device}.")

Using device: cuda.


## Build configurations

In [3]:
# Set config to be equivalent architecture of kaparthy benchmark, however they are not comparable tasks.
@dataclass
class DemoConfig:
    block_size: int = 128        # what is the maximum context length for predictions?
    n_layer: int = 6
    n_head: int = 6
    n_embd: int = 384
    bias: bool = True
    attention_type: str = "global"    
    dropout: float = 0.0
    unk_freq_threshold: float = 0.0
    SurvLayer = "Competing-Risk"                                  # "Competing-Risk"
    tokens_for_univariate_regression = None

config = DemoConfig()

@dataclass
class OptConfig:
    batch_size: int = 64
    eval_interval: int = 1
    learning_rate: float = 3e-4
    epochs: int = 100
    
opt = OptConfig()

## Create data loader on a reduced cohort

In [4]:
from CPRD.data.database import queries

# Get a list of patients which fit a reduced set of criterion
PATH_TO_DB = "/rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModel/preprocessing/processed/cprd.db"
conn = sqlite3.connect(PATH_TO_DB)
cursor = conn.cursor()
# identifiers1 = queries.query_measurement(["bmi", "diastolic_blood_pressure"], cursor)        
identifiers2 = queries.query_diagnosis(["DEPRESSION", "TYPE1DM", "TYPE2DIABETES"], cursor)    #  "DEPRESSION"  ,  "ANXIETY"
# all_identifiers = list(set(identifiers1).intersection(identifiers2))    # Turn smaller list into the set
all_identifiers = identifiers2

if False:
    # Lets take only the first N for faster run-time
    N = np.min((len(all_identifiers), 20000))
    print(f"Using N={N} random samples, from the available {len(all_identifiers)}")
    identifiers = random.choices(all_identifiers, k=N)
else:
    print(f"Using all available {len(all_identifiers)} samples")
    identifiers = all_identifiers

# Build 
dm = FoundationalDataModule(identifiers=identifiers,
                            tokenizer="tabular",
                            batch_size=opt.batch_size,
                            max_seq_length=config.block_size,
                            unk_freq_threshold=config.unk_freq_threshold,
                            include_measurements=True,
                            include_diagnoses=True,
                            preprocess_measurements=True
                           )


vocab_size = dm.train_set.tokenizer.vocab_size

print(f"{len(dm.train_set)} training samples")
print(f"{len(dm.val_set)} validation samples")
print(f"{len(dm.test_set)} test samples")
print(f"{vocab_size} vocab elements (including PAD and UNK tokens)")
print(dm.train_set.tokenizer._itos)

INFO:root:Building polars dataset


Using all available 129717 samples


INFO:root:Using measurements
INFO:root:Using test/measurement standardisation method: normalise
INFO:root:Removing measurement and test outliers. Using three deviations from mean as cutoff
INFO:root:Using diagnoses
INFO:root:Dropping samples with no dynamic events
INFO:root:Using tabular tokenizer


107683 training samples
5983 validation samples
5982 test samples
90 vocab elements (including PAD and UNK tokens)
{0: 'PAD', 1: 'UNK', 2: 'diastolic_blood_pressure', 3: 'bmi', 4: 'eosinophil_count', 5: 'basophil_count', 6: 'corrected_serum_calcium_level', 7: 'DEPRESSION', 8: 'serum_level', 9: 'calculated_LDL_cholesterol_level', 10: 'ANXIETY', 11: 'HYPERTENSION', 12: 'TYPE2DIABETES', 13: 'OSTEOARTHRITIS', 14: 'ASTHMA_PUSHASTHMA', 15: 'ATOPICECZEMA', 16: 'ALLERGICRHINITISCONJ', 17: 'ANY_DEAFNESS_HEARING_LOSS', 18: 'aspartate_transam', 19: 'PREVALENT_IBS', 20: 'ALLCA_NOBCC_VFINAL', 21: 'IHD_NOMI', 22: 'ALCOHOLMISUSE', 23: 'CKDSTAGE3TO5', 24: 'blood_urea', 25: 'PERIPHERAL_NEUROPATHY', 26: 'calcium_adjusted_level', 27: 'HYPOTHYROIDISM_DRAFT_V1', 28: 'COPD', 29: 'AF', 30: 'combined_total_vitamin_D2_and_D3_level', 31: 'HF', 32: 'OSTEOPOROSIS', 33: 'PSORIASIS', 34: 'SUBSTANCEMISUSE', 35: 'GOUT', 36: 'MINFARCTION', 37: 'STROKEUNSPECIFIED', 38: 'ALL_DEMENTIA', 39: 'hydroxyvitamin3', 40: 'hydrox

## Standardisation

This was performed automatically across measurements and tests in the dataloader. The standardisation statistics (bias and scale respectively) are given in the dictionary object. 

We define two mappings to simplify notation later

In [5]:
display(dm.standardisation_dict)

standardise = lambda key, v: (v - dm.standardisation_dict[key][0]) / dm.standardisation_dict[key][1]
unstandardise = lambda key, v: (v * dm.standardisation_dict[key][1]) + dm.standardisation_dict[key][0]

print(standardise("bmi", 30))
print(unstandardise("bmi", standardise("bmi", 20)))

{'combined_total_vitamin_D2_and_D3_level': (56.94353322028673,
  29.241759267841857),
 'bmi': (29.629965163503474, 7.013281083178253),
 'hydroxyvitamin3': (52.36982317356912, 30.382475290251843),
 'diastolic_blood_pressure': (78.86937661562213, 11.727257179342669),
 'eosinophil_count': (0.22162065765491276, 0.18957815785763468),
 'brain_natriuretic_peptide_level': (156.9857534246576, 291.8915253494199),
 'creatinine_ratio': (4.435476477683948, 8.185329269651573),
 'hydroxyvitamin2': (3.268025477707002, 2.8054309314024435),
 'corrected_serum_calcium_level': (2.319034923472716, 0.12451622974555095),
 'calculated_LDL_cholesterol_level': (2.5891371173802233, 1.0358897687206567),
 'blood_urea': (6.702299445123703, 4.281261339020057),
 'blood_calcium': (2.334465408805031, 0.1487655865623607),
 'basophil_count': (0.06984214216499786, 0.10630741873638763),
 'aspartate_transam': (26.791031390134528, 18.877776290025214),
 'calcium_adjusted_level': (2.315393830170679, 0.10855075164585075),
 'seru

0.052762014256647304
20.0


## View the frequency of tokens in the extracted data

In [6]:
import polars as pl
pl.Config.set_tbl_rows(vocab_size - 1)
display(dm.tokenizer._event_counts)

EVENT,counts,freq
str,u32,f64
"""UNK""",0,0.0
"""diastolic_bloo…",2226462,0.422762
"""bmi""",865608,0.164362
"""eosinophil_cou…",811240,0.154039
"""basophil_count…",531602,0.100941
"""corrected_seru…",148788,0.028252
"""DEPRESSION""",86494,0.016424
"""serum_level""",75845,0.014401
"""calculated_LDL…",69073,0.013116


In [7]:
# Extract the measurements, using the fact that the diagnoses are all up upper case. This is needed for automatically setting the configuration below
measurements_for_univariate_regression = [record for record in dm.tokenizer._event_counts["EVENT"] if record.upper() != record]

print(measurements_for_univariate_regression)
print(dm.encode(measurements_for_univariate_regression))
print(dm.decode([7,4,3,2]))

['diastolic_blood_pressure', 'bmi', 'eosinophil_count', 'basophil_count', 'corrected_serum_calcium_level', 'serum_level', 'calculated_LDL_cholesterol_level', 'aspartate_transam', 'blood_urea', 'calcium_adjusted_level', 'combined_total_vitamin_D2_and_D3_level', 'hydroxyvitamin3', 'hydroxyvitamin2', 'creatinine_ratio', 'brain_natriuretic_peptide_level', 'blood_calcium']
[2, 3, 4, 5, 6, 8, 9, 18, 24, 26, 30, 39, 40, 57, 72, 83]
DEPRESSION eosinophil_count bmi diastolic_blood_pressure


## Create models and train

In [8]:
models, m_names = [], []

# My development model
for surv_layer in ["Competing-Risk"]: #, "Single-Risk"]:
    
    ## Create configuration
    config = DemoConfig()
    # Specify which survival head layer to use
    config.SurvLayer = surv_layer   
    # list of univariate measurements to model with Normal distribution
    config.tokens_for_univariate_regression = dm.encode(measurements_for_univariate_regression) 
    
    models.append(SurvStreamGPTForCausalModelling(config, vocab_size).to(device))
    m_names.append(f"SurvStreamGPTForCausalModelling: {surv_layer}")

INFO:root:Using Temporal Positional Encoding. This module uses the patient's age at an event within their time series.
INFO:root:Using Single-Risk DeSurvival head. This module predicts a separate survival curve for each possible future event
INFO:root:Internally scaling time in survival head by 1825 days
INFO:root:In generation forwarding DeSurv on the grid between [0.0, 1825.0], with delta=1/300
INFO:root:ModuleDict(
  (Token 2): Linear(in_features=384, out_features=2, bias=True)
  (Token 3): Linear(in_features=384, out_features=2, bias=True)
  (Token 4): Linear(in_features=384, out_features=2, bias=True)
  (Token 5): Linear(in_features=384, out_features=2, bias=True)
  (Token 6): Linear(in_features=384, out_features=2, bias=True)
  (Token 8): Linear(in_features=384, out_features=2, bias=True)
  (Token 9): Linear(in_features=384, out_features=2, bias=True)
  (Token 18): Linear(in_features=384, out_features=2, bias=True)
  (Token 24): Linear(in_features=384, out_features=2, bias=True)


In [9]:
loss_curves_train = [[] for _ in models]
loss_curves_train_clf = [[] for _ in models]
loss_curves_train_surv = [[] for _ in models]
loss_curves_train_values = [[] for _ in models]

loss_curves_val = [[] for _ in models]
loss_curves_val_clf = [[] for _ in models]
loss_curves_val_surv = [[] for _ in models]
loss_curves_val_values = [[] for _ in models]

In [10]:
for m_idx, (model, m_name) in enumerate(zip(models, m_names)):
    
    print(f"Training model `{m_name}`, with {sum(p.numel() for p in model.parameters())/1e6} M parameters")
    model = model.to(device)

    # create a PyTorch optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=opt.learning_rate)

    best_val, epochs_since_best = np.inf, 0
    for epoch in range(opt.epochs):
        
        epoch_loss, epoch_surv_loss, epoch_values_loss = 0, 0, 0
        model.train()
        for i, batch in enumerate(dm.train_dataloader()):
            # print(i)
            # evaluate the loss
            _, (losses_desurv, loss_values), loss = model(batch['tokens'].to(device), 
                                                          ages=batch['ages'].to(device), 
                                                          values=batch['values'].to(device),
                                                          attention_mask=batch['attention_mask'].to(device)   
                                                          )
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

            # record
            epoch_loss += loss.item()            
            epoch_surv_loss += torch.sum(losses_desurv).item()
            epoch_values_loss += loss_values.item()
        
        epoch_loss /= i
        epoch_surv_loss /= i
        epoch_values_loss /= i
        loss_curves_train[m_idx].append(epoch_loss)
        loss_curves_train_surv[m_idx].append(epoch_surv_loss)
        loss_curves_train_values[m_idx].append(epoch_values_loss)

        # evaluate the loss on val set
        with torch.no_grad(): 
            model.eval()
            if epoch % opt.eval_interval == 0 or epoch == opt.epochs - 1:
                val_loss, val_surv_loss, val_values_loss = 0, 0, 0
                for j, batch in enumerate(dm.val_dataloader()):
                    _, (losses_desurv, loss_values), loss = model(batch['tokens'].to(device), 
                                                                  ages=batch['ages'].to(device),
                                                                  values=batch['values'].to(device),
                                                                  attention_mask=batch['attention_mask'].to(device)   
                                                                  )
                    # record
                    val_loss += loss.item()                    
                    val_surv_loss += torch.sum(losses_desurv).item()
                    val_values_loss += loss_values.item()
                    
                val_loss /= j
                val_surv_loss /= j
                val_values_loss /= j
                loss_curves_val[m_idx].append(val_loss)
                loss_curves_val_surv[m_idx].append(val_surv_loss)
                loss_curves_val_values[m_idx].append(val_values_loss)

                print(f"Epoch {epoch}:\tTrain loss {epoch_loss:.2f}: ({epoch_surv_loss:.2f}, {epoch_values_loss:.2f}). Val loss {val_loss:.2f}: ({val_surv_loss:.2f}, {val_values_loss:.2f})")          
                # TODO: Note not fully accurate as last batch is likely not the same size, will be fixed with lightning
        
            if val_loss >= best_val:
                epochs_since_best += 1
                if epochs_since_best >= 5:
                    break
            else:
                best_val = val_loss
                epochs_since_best = 0

    # Test trained model with a prompt
    # ----------------    
    # set context: diagnosis of depression at 20 years old
    tokens = torch.from_numpy(np.array(dm.encode(["DEPRESSION"])).reshape((1,-1))).to(device)
    ages = torch.tensor([[20*365]], device=device)
    values = torch.tensor([[torch.nan]], device=device)
    
    # generate: sample the next 10 tokens
    new_tokens, new_ages, new_values = model.generate(tokens, ages, values, max_new_tokens=10)
    generated = dm.decode(new_tokens[0].tolist())
    # report:
    for _cat, _age, _value in zip(generated.split(" "), new_ages[0, :], new_values[0, :]):
        try:
            _value = unstandardise(_cat, _value)
        except:
            pass
        print(f"{_cat}".ljust(50) + f"{_value:.02f}".ljust(15) + f"at age {_age/365:.0f} ({_age:.1f} days)")    # with value {_value}

Training model `SurvStreamGPTForCausalModelling: Competing-Risk`, with 10.790921 M parameters
Epoch 0:	Train loss -0.08: (-0.45, 0.29). Val loss -0.19: (-0.66, 0.27)
Epoch 1:	Train loss -0.25: (-0.76, 0.27). Val loss -0.26: (-0.80, 0.29)
Epoch 2:	Train loss -0.31: (-0.88, 0.27). Val loss -0.32: (-0.91, 0.27)
Epoch 3:	Train loss -0.36: (-0.97, 0.26). Val loss -0.31: (-0.90, 0.27)
Epoch 4:	Train loss -0.38: (-1.03, 0.26). Val loss -0.36: (-1.01, 0.29)
Epoch 5:	Train loss -0.38: (-1.04, 0.28). Val loss -0.39: (-1.05, 0.27)
Epoch 6:	Train loss -0.41: (-1.09, 0.26). Val loss -0.41: (-1.08, 0.27)
Epoch 7:	Train loss -0.41: (-1.10, 0.27). Val loss -0.41: (-1.09, 0.27)
Epoch 8:	Train loss -0.44: (-1.15, 0.27). Val loss -0.42: (-1.13, 0.28)
Epoch 9:	Train loss -0.45: (-1.18, 0.27). Val loss -0.46: (-1.18, 0.27)
Epoch 10:	Train loss -0.48: (-1.21, 0.26). Val loss -0.47: (-1.21, 0.27)
Epoch 11:	Train loss -0.47: (-1.22, 0.27). Val loss -0.46: (-1.21, 0.28)
Epoch 12:	Train loss -0.49: (-1.25, 0.27

## Comparing output to real data

In [11]:
for batch in dm.train_dataloader():
    break
    
conditions = batch["tokens"].numpy().tolist()
# delta_ages = batch["ages"][:, 1:] - batch["ages"][:, :-1]
for idx, (token, _age, _value) in enumerate(zip(conditions[0], batch["ages"][0,:],  batch["values"][0,:])):
    if token == 0 or idx >= 10:
        break
    _cat = dm.decode([token])
    try:
        _value = unstandardise(_cat, _value)
    except:
        pass
        
    print(f"{_cat}".ljust(50) + f"{_value:.02f}".ljust(15) + f"at age {_age/365:.0f} ({_age:.1f} days)")

SUBSTANCEMISUSE                                   nan            at age 19 (7109.0 days)
DEPRESSION                                        nan            at age 21 (7840.0 days)
ANXIETY                                           nan            at age 21 (7840.0 days)
bmi                                               29.50          at age 23 (8429.0 days)
diastolic_blood_pressure                          70.00          at age 23 (8429.0 days)
eosinophil_count                                  0.20           at age 23 (8429.0 days)
diastolic_blood_pressure                          68.00          at age 23 (8450.0 days)
diastolic_blood_pressure                          62.00          at age 23 (8471.0 days)
diastolic_blood_pressure                          65.00          at age 23 (8479.0 days)
diastolic_blood_pressure                          60.00          at age 23 (8485.0 days)


In [12]:
cols = ["k", "r", "b", "y"]

# Plot loss
plt.figure()
for m_idx, _ in enumerate(models):
    # Training
    iterations = np.linspace(0, len(loss_curves_train[m_idx]), len(loss_curves_train[m_idx])) * opt.eval_interval
    plt.plot(iterations, loss_curves_train[m_idx], label=f"{m_names[m_idx]}-train", c=cols[m_idx], linestyle='dashed')
    # Validation
    iterations = np.linspace(0, len(loss_curves_val[m_idx]), len(loss_curves_val[m_idx])) * opt.eval_interval
    plt.plot(iterations, loss_curves_val[m_idx], label=f"{m_names[m_idx]}-val", c=cols[m_idx])
plt.legend()
plt.savefig(f"figs/competing_risk/loss.png")

# Plot DeSurv loss
plt.figure()
for m_idx, _ in enumerate(models):
    # Training
    iterations = np.linspace(0, len(loss_curves_train_surv[m_idx]), len(loss_curves_train_surv[m_idx])) * opt.eval_interval
    plt.plot(iterations, loss_curves_train_surv[m_idx], label=f"{m_names[m_idx]}-train", c=cols[m_idx], linestyle='dashed')
    # Validation
    iterations = np.linspace(0, len(loss_curves_val_surv[m_idx]), len(loss_curves_val_surv[m_idx])) * opt.eval_interval
    plt.plot(iterations, loss_curves_val_surv[m_idx], label=f"{m_names[m_idx]}-val", c=cols[m_idx])
plt.legend()
plt.savefig(f"figs/competing_risk/loss_desurv.png")

plt.figure()
for m_idx, _ in enumerate(models):
    # Training
    iterations = np.linspace(0, len(loss_curves_train_values[m_idx]), len(loss_curves_train_values[m_idx])) * opt.eval_interval
    plt.plot(iterations, loss_curves_train_values[m_idx], label=f"{m_names[m_idx]}-train", c=cols[m_idx], linestyle='dashed')
    # Validation
    iterations = np.linspace(0, len(loss_curves_val_values[m_idx]), len(loss_curves_val_values[m_idx])) * opt.eval_interval
    plt.plot(iterations, loss_curves_val_values[m_idx], label=f"{m_names[m_idx]}-val", c=cols[m_idx])
plt.legend()
plt.savefig(f"figs/competing_risk/loss_val.png")

# Prompt testing

## Diabetes: How related conditions are impacted by each other
Probability of type II diabetes before and after a type I diagnosis

In [13]:
t1_token = dm.tokenizer._stoi["TYPE1DM"]
t2_token = dm.tokenizer._stoi["TYPE2DIABETES"]


base_prompt = ["DEPRESSION"]
ages_in_years = [20]
base_values = [torch.tensor([torch.nan])]

to_days = lambda a_list: torch.FloatTensor([365 * _a for _a in a_list]).reshape((1,-1)).to(device)

# Create a set of prompts
prompts, ages, values, desc = [], [], [], []
# control prompt
desc.append("Depression")
prompts.append(base_prompt)
ages.append(ages_in_years)
values.append(base_values)
# prompt with type 1 diabetes
desc.append("Depression -> Type 1")
prompts.append(base_prompt + ["TYPE1DM"])
ages.append(ages_in_years + [21])
values.append(base_values + [torch.tensor([torch.nan])])

desc.append("Depression - > Type 2")
prompts.append(base_prompt + ["TYPE2DIABETES"])
ages.append(ages_in_years + [21])
values.append(base_values + [torch.tensor([torch.nan])])

for model_idx, model in enumerate(models):
    print(f"\n\n{m_names[model_idx]}\n--------------------------------------")
    with torch.no_grad(): 
        model.eval()

        prompt_survs = []
        for p_idx, (prompt, age, value) in enumerate(zip(prompts, ages, values)):
            print(f"\n{desc[p_idx]}: \t ({','.join(prompt)}): ")
            encoded_prompt = torch.from_numpy(np.array(dm.encode(prompt)).reshape((1,-1))).to(device)
            (surv, val_dist), _, _ = model(encoded_prompt,
                                           values=torch.tensor(value).to(device),
                                           ages=to_days(age),
                                           is_generation=True)
            prompt_survs.append(surv)

        for si, _ in enumerate(surv):
            plt.close()
            event_name = dm.decode([si + 1])
            for p_idx in range(len(prompts)):
                plt.plot(model.surv_layer.t_eval / 365, prompt_survs[p_idx][si][0, :], label=f"{desc[p_idx]}")
            plt.legend()
            plt.savefig(f"figs/competing_risk/diabetes/{event_name}.png")




SurvStreamGPTForCausalModelling: Competing-Risk
--------------------------------------

Depression: 	 (DEPRESSION): 

Depression -> Type 1: 	 (DEPRESSION,TYPE1DM): 

Depression - > Type 2: 	 (DEPRESSION,TYPE2DIABETES): 


## Values: How increasing BMI affects diagnosis risk

In [14]:
events_of_interest = ["bmi", "diastolic_blood_pressure", 
                      "TYPE1DM", "TYPE2DIABETES",
                      "HYPERTENSION", "OSTEOARTHRITIS",
                      "CKDSTAGE3TO5",
                      "HF", "ISCHAEMICSTROKE"
                     ]
prompt = ["bmi"]
values = [torch.tensor([standardise(_cat, v) for _cat in prompt], device=device) for v in [12.,15.,18.,21.,24.,30.,40.]]
age = [40]

for model_idx, model in enumerate(models):
    print(f"\n\n{m_names[model_idx]}\n--------------------------------------")
    with torch.no_grad(): 
        model.eval()

        prompt_survs = []
        for p_idx, value in enumerate(values):
            print(f"Value {value}\n======")
            encoded_prompt = torch.from_numpy(np.array(dm.encode(prompt)).reshape((1,-1))).to(device)
            (surv, val_dist), _, _ = model(encoded_prompt,
                                           values=value,
                                           ages=to_days(age),
                                           is_generation=True)
            prompt_survs.append(surv)

        for si, _ in enumerate(surv):
            plt.close()
            event_name = dm.decode([si + 1])
            
            if event_name in events_of_interest:
                
                for p_idx in range(len(prompt_survs)):
                    bmi_value = unstandardise("bmi", values[p_idx])
                    plt.plot(model.surv_layer.t_eval / 365, prompt_survs[p_idx][si][0, :], label=f"BMI {bmi_value.item():.2f}")
                plt.xlabel("t (years)")
                plt.ylabel("P(T>t) ()")
                plt.legend()
                plt.savefig(f"figs/competing_risk/bmi/{event_name}.png")




SurvStreamGPTForCausalModelling: Competing-Risk
--------------------------------------
Value tensor([-2.5138], device='cuda:0')
Value tensor([-2.0860], device='cuda:0')
Value tensor([-1.6583], device='cuda:0')
Value tensor([-1.2305], device='cuda:0')
Value tensor([-0.8028], device='cuda:0')
Value tensor([0.0528], device='cuda:0')
Value tensor([1.4786], device='cuda:0')


## Values: How increasing diastolic_blood_pressure affects likelihood of diagnoses

In [15]:
events_of_interest = ["bmi", "diastolic_blood_pressure", 
                      "TYPE1DM", "TYPE2DIABETES",
                      "HYPERTENSION", "OSTEOARTHRITIS",
                      "CKDSTAGE3TO5",
                      "HF", "ISCHAEMICSTROKE"
                     ]

prompt = ["diastolic_blood_pressure"]
values = [torch.tensor([standardise(_cat, _value) for _cat in prompt], device=device) for _value in [60.,70.,80.,90.,100.,120.]]
age = [40]


for model_idx, model in enumerate(models):
    print(f"\n\n{m_names[model_idx]}\n--------------------------------------")
    with torch.no_grad(): 
        model.eval()

        prompt_survs = []
        for p_idx, value in enumerate(values):
            print(f"Value {value}\n======")
            encoded_prompt = torch.from_numpy(np.array(dm.encode(prompt)).reshape((1,-1))).to(device)
            (surv, val_dist), _, _ = model(encoded_prompt,
                                           values=value,
                                           ages=to_days(age),
                                           is_generation=True)
            prompt_survs.append(surv)

        for si, _ in enumerate(surv):
            plt.close()
            event_name = dm.decode([si + 1])
            
            if event_name in events_of_interest:
                
                for p_idx in range(len(prompt_survs)):
                    dbp_value = unstandardise("diastolic_blood_pressure", values[p_idx])
                    plt.plot(model.surv_layer.t_eval / 365, prompt_survs[p_idx][si][0, :], label=f"DBP {dbp_value.item():.2f}")
                plt.xlabel("t (years)")
                plt.ylabel("P(T>t) ()")
                plt.legend()
                plt.savefig(f"figs/competing_risk/diastolic_blood_pressure/{event_name}.png")




SurvStreamGPTForCausalModelling: Competing-Risk
--------------------------------------
Value tensor([-1.6090], device='cuda:0')
Value tensor([-0.7563], device='cuda:0')
Value tensor([0.0964], device='cuda:0')
Value tensor([0.9491], device='cuda:0')
Value tensor([1.8018], device='cuda:0')
Value tensor([3.5073], device='cuda:0')


## Values: How varying diagnosis affects value of diastolic_blood_pressure

In [16]:
measurements_of_interest = ["diastolic_blood_pressure"]
t1_token = dm.tokenizer._stoi["diastolic_blood_pressure"]

diagnoses = [["DEPRESSION"],["TYPE2DIABETES"], ["HF"], ["HYPERTENSION"]]
values = torch.tensor([torch.nan], device=device)
age = [40]

for model_idx, model in enumerate(models):
    print(f"\n\n{m_names[model_idx]}\n--------------------------------------")
    with torch.no_grad(): 
        model.eval()

    for p_idx, diagnosis in enumerate(diagnoses):
        print(f"\nDiagnosis {diagnosis}\n======")
        encoded_prompt = torch.from_numpy(np.array(dm.encode(diagnosis)).reshape((1,-1))).to(device)
        (surv, val_dist), _, _ = model(encoded_prompt,
                                       values=values,
                                       ages=to_days(age),
                                       is_generation=True)
        dist = val_dist[model.value_layer.token_key(t1_token)]
        print(f"standardised diastolic_blood_pressure ~ N({dist.loc.item():.1f}, {dist.scale.item():.1f})")





SurvStreamGPTForCausalModelling: Competing-Risk
--------------------------------------

Diagnosis ['DEPRESSION']
standardised diastolic_blood_pressure ~ N(-0.1, 0.9)

Diagnosis ['TYPE2DIABETES']
standardised diastolic_blood_pressure ~ N(0.0, 0.9)

Diagnosis ['HF']
standardised diastolic_blood_pressure ~ N(-0.0, 1.0)

Diagnosis ['HYPERTENSION']
standardised diastolic_blood_pressure ~ N(0.4, 1.0)


## Values: How increasing bmi affects value of diastolic_blood_pressure

In [17]:
t1_token = dm.tokenizer._stoi["diastolic_blood_pressure"]

prompt = ["bmi"]
values = [torch.tensor([standardise(_cat, _value) for _cat in prompt], device=device) for _value in [12.,15.,18.,21.,24.,30.,40.,50.]]
age = [40]

for model_idx, model in enumerate(models):
    print(f"\n\n{m_names[model_idx]}\n--------------------------------------")

    for p_idx, value in enumerate(values):
        print(f"\nValues {value.tolist()}\n======")
        encoded_prompt = torch.from_numpy(np.array(dm.encode(prompt)).reshape((1,-1))).to(device)
        (surv, val_dist), _, _ = model(encoded_prompt,
                                       values=value,
                                       ages=to_days(age),
                                       is_generation=True)
        
        dist = val_dist[model.value_layer.token_key(t1_token)]
        print(f"standardised diastolic_blood_pressure ~ N({dist.loc.item():.1f}, {dist.scale.item():.1f})")
        # print(f"\tprobability of type II diabetes: {100*float(probs[0, 0, t2_token].cpu().detach().numpy()):.4f}%")



SurvStreamGPTForCausalModelling: Competing-Risk
--------------------------------------

Values [-2.5137970447540283]
standardised diastolic_blood_pressure ~ N(-0.9, 0.9)

Values [-2.0860371589660645]
standardised diastolic_blood_pressure ~ N(-0.8, 0.8)

Values [-1.6582773923873901]
standardised diastolic_blood_pressure ~ N(-0.6, 0.8)

Values [-1.2305175065994263]
standardised diastolic_blood_pressure ~ N(-0.5, 0.8)

Values [-0.8027576804161072]
standardised diastolic_blood_pressure ~ N(-0.3, 0.8)

Values [0.05276201292872429]
standardised diastolic_blood_pressure ~ N(0.1, 0.8)

Values [1.478628158569336]
standardised diastolic_blood_pressure ~ N(0.6, 0.9)

Values [2.904494285583496]
standardised diastolic_blood_pressure ~ N(0.5, 1.0)


# Appendix: model architectures

In [18]:
for model_idx, model in enumerate(models):
    print(f"\n\n{m_names[model_idx]}\n" + "="*len(m_names[model_idx]))
    print(f"\n\n{model}")



SurvStreamGPTForCausalModelling: Competing-Risk


SurvStreamGPTForCausalModelling(
  (transformer): TTETransformer(
    (wpe): TemporalPositionalEncoding()
    (wte): DataEmbeddingLayer(
      (token_embed_layer): Embedding(90, 384, padding_idx=0)
      (value_embed_layer): EmbeddingBag(90, 384, mode=sum, padding_idx=0)
    )
    (drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadedSelfAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=384, out_features=384, bias=False)
          (v_proj): Linear(in_features=384, out_features=384, bias=False)
          (q_proj): Linear(in_features=384, out_features=384, bias=False)
          (out_proj): Linear(in_features=384, out_features=384, bias=True)
        )
        (ln_2): LayerNorm((384,), eps=1e-05, element

In [19]:
!jupyter nbconvert --to html --no-input competing_risk.ipynb

[NbConvertApp] Converting notebook competing_risk.ipynb to html
[NbConvertApp] Writing 600727 bytes to competing_risk.html
